In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
treatment_option = 'vaso'
observation_window = 30
step = 3

In [17]:
static_feat = ['age', 'is_female', 'is_male',
       'race_white', 'race_black', 'race_hispanic', 'race_other',
       'metastatic_cancer', 'diabetes', 'height', 'weight', 'bmi']

var = ['hemoglobin','crp','heartrate','creatinine',
 'hematocrit','sysbp','tempc','pt','sodium','diasbp', 'gcs','platelet','ptt',
 'chloride','resprate','glucose','bicarbonate','bands', 'bun',
 'magnesium','urineoutput','inr','lactate','aniongap','spo2','wbc','meanbp']

In [30]:
full_df = pd.read_csv('target_full_data.csv')

In [66]:
n_X_static_features = 12
n_X_features = len(var)

In [193]:
n_X_features

27

### The ICU stay id with enough data (33 hour 11 interval)

In [31]:
max_interval_data = full_df.groupby('icustay_id').agg({'interval_index': max}).reset_index()
max_interval_data[max_interval_data.interval_index == 10]

,icustay_id,interval_index
0,200003,10.0
1,200019,10.0
2,200030,10.0
3,200036,10.0
4,200059,10.0
...,...,...
10062,299969,10.0
10063,299988,10.0
10064,299992,10.0
10065,299995,10.0


In [50]:
check_id = max_interval_data[max_interval_data.interval_index == 10].icustay_id.unique()

In [52]:
used_id = []
for ii in check_id:
    tmp = full_df[full_df.icustay_id == ii]
    if tmp.shape[0] == 11:
        used_id.append(ii)

In [53]:
len(used_id)

9064

In [77]:
full_df = full_df.fillna(full_df.mean())

### Static data

,age,is_female,is_male,race_white,race_black,race_hispanic,race_other,metastatic_cancer,diabetes,height,weight,bmi
0,48.2940,0,1,1,0,0,0,0,0,NaN,77.0,NaN
1,82.8820,1,0,1,0,0,0,0,0,NaN,65.0,NaN
2,54.1915,0,1,0,1,0,0,0,0,187.96,113.6,32.154922
3,74.9339,0,1,1,0,0,0,0,0,NaN,79.0,NaN
4,78.8984,1,0,1,0,0,0,0,0,NaN,54.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
10062,75.4760,0,1,1,0,0,0,0,0,177.80,66.9,21.162287
10063,32.7947,0,1,1,0,0,0,0,0,172.72,90.0,30.168745
10064,41.5916,0,1,1,0,0,0,0,0,185.42,90.0,26.177572
10065,23.3763,0,1,1,0,0,0,0,0,NaN,70.4,NaN


In [195]:
X_static = np.random.normal(0, 0.5, size=(len(used_id),n_X_static_features))
for j, ID in enumerate(used_id):
    if os.path.exists('static/%d.static.npy' % ID):
        X_static[j] = np.load('static/%d.static.npy' % ID)
    else:
        print(ID)
X_static = np.nan_to_num(X_static)

### X data

In [197]:
X = []
for ii, ID in tqdm(enumerate(used_id)):
    tmp = full_df[full_df.icustay_id == ID]
    X.append(tmp[var][:-1].to_numpy())

9064it [00:07, 1164.27it/s]


In [198]:
X_new = np.zeros(shape=(len(used_id), observation_window//3, n_X_features))
for i in range(len(used_id)):
    X_new[i] = X[i]

### Treatment

In [199]:
treatment = pd.read_csv('vas_treatment_index.csv')

In [200]:
treatment[treatment['icustay_id'] == 243255].start_interval_index.isnull()

10062    True
Name: start_interval_index, dtype: bool

In [201]:
A = np.zeros(shape=(len(used_id), observation_window//step))
for ii, ID in tqdm(enumerate(used_id)):
    tmp = treatment[treatment['icustay_id'] == ID]
    if tmp.start_interval_index.isnull().values:
        A[ii, :] = np.zeros(observation_window//step)
    else:
        res = np.zeros(observation_window//step)
        start_ind, end_ind = int(tmp.start_interval_index.values[0]), int(tmp.end_interval_index.values[0])
        i = start_ind
        while i <= end_ind:
            if end_ind >= 10:
                break
            res[i] = 1
            i += 1
        A[ii, :] = res

9064it [00:03, 2620.83it/s]


### Simulation

In [202]:
X_mean = np.mean(X_new, axis=(0,1))
X_std = np.std(X_new, axis=(0,1))

X_static_mean = np.mean(X_static, axis=0)
X_static_std = np.std(X_static, axis=0)

X_norm = np.zeros(shape=(len(used_id),observation_window//step,n_X_features))
X_static_norm = np.zeros(shape=(len(used_id),n_X_static_features))


for i in range(observation_window//step):
    for j in range(n_X_features):
        X_norm[:,i,j] = (X_new[:,i,j]-X_mean[j])/X_std[j]

for c in range(n_X_static_features):
    if c in (0, 8, 9, 10):
        X_static_norm[:, c] = (X_static[:,c]-X_static_mean[c])/X_static_std[c]
    else:
        X_static_norm[:, c] = X_static[:,c]

In [203]:
N_treated = len(np.where(np.sum(A, axis=1)>0)[0])

A_final = np.where(np.sum(A, axis=1)>0, 1, 0)

all_idx = np.arange(len(used_id))
np.random.shuffle(all_idx)

train_ratio = 0.7
val_ratio = 0.1

train_idx = all_idx[:int(len(all_idx)*train_ratio)]
val_idx = all_idx[int(len(all_idx) * train_ratio):int(len(all_idx) * train_ratio)+int(len(all_idx) * val_ratio)]
test_idx = all_idx[int(len(all_idx) * train_ratio)+int(len(all_idx) * val_ratio):]

train_ids = [used_id[x] for x in train_idx]
val_ids = [used_id[x] for x in val_idx]

In [225]:
split = []
for x in used_id:
    if x in train_ids:
        split.append([x, 1])
    elif x in val_ids:
        split.append([x, 2])
    else:
        split.append([x, 0])


# num of P
p = 5
# weight of hidden confounders
gamma_h = 0.7
# num of hidden
h = 1
N_treated = len(np.where(np.sum(A, axis=1)>0)[0])
N = len(used_id)

In [226]:
eta, epsilon = np.random.normal(0, 0.001, size=(N, observation_window//step,n_X_features)),np.random.normal(0,0.001, size=(N,observation_window//step,h))
delta = np.random.uniform(-1, 1, size=(n_X_features+ n_X_static_features, h))

A_final = np.where(np.sum(A, axis=1)>0, 1, 0)
Z = np.random.normal(0, 0.5, size=(N,h))
Z[np.where(np.sum(A, axis=1)>0), :] = np.random.normal(1, 0.5, size=(N_treated, h))
Z_all = [Z]
Q_all = []

In [227]:
for t in range(1, observation_window//step+1):
    i = 1
    tmp_x = 0
    tmp_z = 0
    while (t-i) >= 0 and i <= p:

        mu = np.random.normal(1 - (i / p), (1 / p), size=(N, h))
        v = np.random.normal(0, 0.02, size=(N, h))
        v[np.where(np.sum(A, axis=1) > 0), :] = np.random.normal(1, 0.02, size=(N_treated, h))
        tmp_z += np.multiply(mu, Z_all[t - i]) + np.multiply(v, np.tile(A[:, t - i], (h, 1)).T)

        i += 1
    X_sample = X_norm[:,t-1,:]
    Z = tmp_z/(i-1) + epsilon[:,t-1,:]
    Z_all.append(Z)
    Q = gamma_h * Z + (1 - gamma_h) * np.expand_dims(np.mean(np.concatenate((X_sample, X_static_norm), axis=1), axis=1), axis=1)
    Q_all.append(Q)

In [228]:
w = np.random.uniform(-1, 1, size=(2, 1))
b = np.random.normal(0, 0.1, size=(N, 1))
Y_f = np.matmul(np.concatenate((Q, np.expand_dims(A_final, axis=1)),axis=1), w) + b

w = np.random.uniform(-1, 1, size=(2, 1))
b = np.random.normal(0, 0.1, size=(N, 1))
A_final_cf = np.where(A_final==1, 0, 1)
Y_cf = np.matmul(np.concatenate((Q, np.expand_dims(A_final_cf, axis=1)),axis=1), w) + b

Y_f_norm = Y_f
Y_cf_norm = Y_cf

In [229]:
data_synthetic = 'data_synthetic'
dir_target = '{}/data_mimic_mean_syn_{}'.format(data_synthetic, gamma_h)
dir_base = '{}/data_baseline_mimic_mean_syn_{}'.format(data_synthetic,gamma_h)

os.makedirs(dir_target, exist_ok=True)
os.makedirs(dir_base, exist_ok=True)

In [230]:
for n in tqdm(range(len(used_id))):
    x = np.zeros(shape=(observation_window//step, n_X_features))
    ID = used_id[n]
    out_x_file = '{}/{}.x.npy'.format(dir_target, ID)
    out_static_file = '{}/{}.static.npy'.format(dir_target, ID)
    out_a_file = '{}/{}.a.npy'.format(dir_target, ID)
    out_y_file = '{}/{}.y.npy'.format(dir_target, ID)
    for t in range(observation_window//step):
        x[t, :] = X_norm[n,t,:]
    x_static = X_static_norm[n,:]
    a = A[n,:]

    y = [Y_f_norm[n], Y_cf_norm[n]]

    np.save(out_x_file, x)
    np.save(out_static_file, x_static)
    np.save(out_a_file, a)
    np.save(out_y_file, y)

100%|████████████████████████████████████████████████████████████| 9064/9064 [00:10<00:00, 881.71it/s]


In [231]:
n_classes = 1
for t in tqdm(range(observation_window//step)):
    # a + y_f + y_cf + n_covariates + split

    out_matrix = np.zeros((len(used_id), n_X_features+n_X_static_features+1+n_classes*2+1))

    out_matrix[:,0] = A_final
    out_matrix[:,(1+n_classes*2):(1+n_classes*2+n_X_features)] = X_norm[:,t,:]
    out_matrix[:,(1+n_classes*2+n_X_features):(1+n_classes*2+n_X_features+n_X_static_features)] = X_static_norm

    out_matrix[:, 1:2] = Y_f_norm
    out_matrix[:, 2:3] = Y_cf_norm

    out_matrix[:,-1] = np.array(split)[:,-1]

    df = pd.DataFrame(out_matrix)
    df.to_csv('{}/{}.csv'.format(dir_base, t+1), index=False)


df = pd.DataFrame(np.array(split))
df.to_csv('{}/train_test_split.csv'.format(dir_target), index=False, header=False)

100%|█████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.74it/s]
